<a href="https://colab.research.google.com/github/jvishnuvardhan/Medium_Articles/blob/master/Save_weights_before_after_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install tensorflow==2.2

In [10]:
# Import modules
import tensorflow as tf
from tensorflow import keras
import os, datetime
import numpy as np

Epochs = 10
# Load data
mnist = tf.keras.datasets.mnist

# Split the data to train and test
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Process the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define model
def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='Flatten'),
    tf.keras.layers.Dense(512, activation='relu', name = 'Dense1'),
    tf.keras.layers.Dropout(0.2, name = 'Dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name = 'Output')
    ])
  
  return model

# Create a basic model instance
model = create_model()

# Compile the model
model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy'])



In [12]:
# Saving weights before training (tf format)
model.save_weights('My_weights_tf',save_format='tf')

# Saving weights before training (h5 format)
model.save_weights('My_weights_h5',save_format='h5')

#prediction
prediction_model = model.predict(tf.reshape(x_test[0],shape=(1,28,28)))
print("Prediction with original model : ", np.argmax(prediction_model))
# Instantiate new model
new_model = create_model() # Initializes new random weights and biases
#print(new_model.weights)

#print('original model')
#print(model.weights) # notice that original model weights and new_model weights are different

#prediction
prediction_new_model = new_model.predict(tf.reshape(x_test[0],shape=(1,28,28)))
print("Prediction with new model : ", np.argmax(prediction_new_model))

# prediction after loading saved weights
new_model.load_weights('My_weights_tf')
prediction_new_model_after_loading_weights = new_model.predict(tf.reshape(x_test[0],shape=(1,28,28)))
print("Prediction with new model (after loading weights) : ", np.argmax(prediction_new_model_after_loading_weights))

Prediction with original model :  0
Prediction with new model :  9
Prediction with new model (after loading weights) :  0


In [13]:
# train the modell
model.fit(x_train, y_train, epochs = 1, validation_data=(x_test, y_test), verbose=1)

# Saving weights after training (tf format)
model.save_weights('My_weights_tf_after_training',save_format='tf')

# Saving weights after training (h5 format)
model.save_weights('My_weights_h5_after_training',save_format='h5')

# evaluate loss and accuracy
loss, acc = model.evaluate(x_test, y_test, verbose=1)
print("Original model, accuracy: {:5.2f}%".format(100*acc))

313/313 [==============================] - 1s 2ms/step - loss: 0.1121 - accuracy: 0.9644
Original model, accuracy: 96.44%


In [16]:
# Instantiate new model
new_model = create_model() # Initializes new random weights and biases

# Compile the model (new_model)
new_model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy'])

# prediction after loading saved weights (after training)
new_model.load_weights('My_weights_tf_after_training')

# evaluate loss and accuracy
loss, acc_afte_loading_weights = new_model.evaluate(x_test, y_test, verbose=1)
print("new_model, accuracy: {:5.2f}%".format(100*acc_afte_loading_weights))

313/313 [==============================] - 1s 2ms/step - loss: 0.1121 - accuracy: 0.9644
new_model, accuracy: 96.44%


In [18]:
# When the model has a custom metric with external params

# Instantiate new model
model2 = create_model() # Initializes new random weights and biases

magic_num = 1 # This is just for demonstrating usage of an external parameter inside custom metric

# Custom Metric (with external parameters)
def metric_with_params(magic_num):
  def metric(yTrue,yPred):
    yTrue = yTrue*magic_num
    return tf.keras.metrics.sparse_categorical_accuracy(yTrue, yPred)
  return metric

# Compile the model
model2.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy', metric_with_params(magic_num)])

# train the modell
model2.fit(x_train, y_train, epochs = 1, validation_data=(x_test, y_test), verbose=1)

# Saving weights after training (tf format)
model2.save_weights('My_weights_custom_metric',save_format='tf')
# evaluate loss and accuracy
loss, acc,acc_cust = model2.evaluate(x_test, y_test, verbose=1)
print("Original model, accuracy: {:5.2f}%".format(100*acc))

313/313 [==============================] - 1s 2ms/step - loss: 0.1045 - accuracy: 0.9687 - metric: 0.9687
Original model, accuracy: 96.87%


In [19]:
# Instantiate new model
model_loaded = create_model() # Initializes new random weights and biases

# Compile the model (new_model)
model_loaded.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy',metric_with_params(magic_num)])

# prediction after loading saved weights (after training)
model_loaded.load_weights('My_weights_custom_metric')

# evaluate loss and accuracy
loss, acc_after_loading,acc_cust = model_loaded.evaluate(x_test, y_test, verbose=1)
print("new_model, accuracy: {:5.2f}%".format(100*acc_after_loading))

313/313 [==============================] - 1s 2ms/step - loss: 0.1045 - accuracy: 0.9687 - metric: 0.9687
new_model, accuracy: 96.87%
